In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'window_low_bound': [2], 'window_high_bound': [5],
                            'first_candle': [0.6], 'second_candle': [0.8]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is sell
Number of combinations is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:04<00:00, 64.36s/it]


# Check local statistics

In [11]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_LinearReg,2,5,0.6,0.8,3,0,932,1.711900,-0.547083,663.490800,-509.881667,1.6525,2.2312,2.3512,2.2908,2.0999,1.9970,1.9471,1.9010,1.8656,1.8334,1.7339,1.6630,1.5792,1.5557,1.5334,1.4897,1.4704,1.4416,1.4074,1.4332,1.4123,1.4119,1.3972,1.3870,-0.33,-0.53,-0.63,-0.64,-0.55,-0.61,-0.64,-0.60,-0.61,-0.7,-0.63,-0.56,-0.49,-0.48,-0.53,-0.52,-0.55,-0.45,-0.43,-0.53,-0.64,-0.48,-0.59,-0.41
1,Pattern_LinearReg,2,6,0.6,0.8,3,0,1055,1.681629,-0.549167,719.118771,-579.370833,1.6012,2.1406,2.2498,2.1849,2.0328,1.9327,1.8949,1.8551,1.8150,1.7838,1.6795,1.6111,1.5405,1.5209,1.4993,1.4673,1.4652,1.4421,1.4290,1.4589,1.4489,1.4469,1.4325,1.4262,-0.32,-0.52,-0.62,-0.63,-0.53,-0.59,-0.62,-0.59,-0.60,-0.7,-0.63,-0.53,-0.47,-0.45,-0.49,-0.49,-0.58,-0.48,-0.48,-0.56,-0.67,-0.52,-0.63,-0.48
2,Pattern_LinearReg,2,7,0.6,0.8,3,0,1182,1.654296,-0.560000,773.377675,-661.920000,1.5956,2.0950,2.1891,2.1204,1.9884,1.9021,1.8632,1.8077,1.7741,1.7500,1.6483,1.5873,1.5175,1.5026,1.4788,1.4584,1.4612,1.4367,1.4271,1.4501,1.4354,1.4198,1.4007,1.3936,-0.33,-0.52,-0.61,-0.63,-0.53,-0.59,-0.61,-0.58,-0.61,-0.7,-0.63,-0.53,-0.49,-0.49,-0.54,-0.51,-0.59,-0.49,-0.49,-0.57,-0.67,-0.57,-0.64,-0.52


# Research parameter influence

In [13]:
param = 'Pattern_window_high_bound'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_window_high_bound,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,3,1.625076,-0.527808,225.588708,-187.060817,18621
1,4,1.523453,-0.451042,390.604569,-336.971425,37876
2,5,1.523395,-0.475033,491.862304,-443.211683,47029
3,6,1.505206,-0.481067,533.726879,-504.731083,53244


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [12]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [2], 'window_high_bound': [5],
                            'first_candle': [0.6], 'second_candle': [0.8]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)